# Import libraries

In [26]:
import datetime as datetime
from feature_extraction.indicator_features import *
from feature_extraction.daily_features import *  
import jalali_pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytse_client as tse
from pytse_client.download import download_financial_indexes
# import plotly.express as px
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# from plotly.offline import init_notebook_mode
import seaborn as sns
from sklearn.model_selection import train_test_split
import ta
from zigzag import peak_valley_pivots

# Import data

In [27]:
tse.download(
    symbols = ['فولاد'], 
    write_to_csv = True, 
    include_jdate = True
)
folad = tse.Ticker(
    symbol = 'فولاد',  
    adjust = True
)
foladHist = folad.history
foladClient = folad.client_types

print(f'Shape of Folad price history: {foladHist.shape}')
print(f"Shape of Folad's client history: {foladClient.shape}")

Shape of Folad price history: (3418, 10)
Shape of Folad's client history: (3036, 18)


In [28]:
print(f'{foladHist.info()}')
foladHist.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3418 entries, 0 to 3417
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       3418 non-null   datetime64[ns]
 1   open       3418 non-null   float64       
 2   high       3418 non-null   float64       
 3   low        3418 non-null   float64       
 4   adjClose   3418 non-null   float64       
 5   value      3418 non-null   int64         
 6   volume     3418 non-null   int64         
 7   count      3418 non-null   int64         
 8   yesterday  3418 non-null   float64       
 9   close      3418 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(3)
memory usage: 267.2 KB
None


,date,open,high,low,adjClose,value,volume,count,yesterday,close
0,2007-03-11,16.0,16.0,16.0,16.0,889437216900,468077431,7736,16.0,16.0
1,2007-03-12,16.0,16.0,16.0,16.0,193879458000,100041000,9214,16.0,16.0
2,2007-03-13,16.0,16.0,16.0,16.0,249241504527,126270939,5862,16.0,16.0
3,2007-03-14,16.0,16.0,16.0,16.0,51666379451,26705128,1901,16.0,16.0
4,2007-03-17,16.0,16.0,16.0,16.0,28239006789,14877283,1514,16.0,16.0


In [29]:
print(f'{foladClient.info()}')
foladClient.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3036 entries, 0 to 3035
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   date                         3036 non-null   object 
 1   individual_buy_count         3036 non-null   object 
 2   corporate_buy_count          3036 non-null   object 
 3   individual_sell_count        3036 non-null   object 
 4   corporate_sell_count         3036 non-null   object 
 5   individual_buy_vol           3036 non-null   object 
 6   corporate_buy_vol            3036 non-null   object 
 7   individual_sell_vol          3036 non-null   object 
 8   corporate_sell_vol           3036 non-null   object 
 9   individual_buy_value         3036 non-null   object 
 10  corporate_buy_value          3036 non-null   object 
 11  individual_sell_value        3036 non-null   object 
 12  corporate_sell_value         3036 non-null   object 
 13  individual_buy_mea

,date,individual_buy_count,corporate_buy_count,individual_sell_count,corporate_sell_count,individual_buy_vol,corporate_buy_vol,individual_sell_vol,corporate_sell_vol,individual_buy_value,corporate_buy_value,individual_sell_value,corporate_sell_value,individual_buy_mean_price,individual_sell_mean_price,corporate_buy_mean_price,corporate_sell_mean_price,individual_ownership_change
0,20220921,2386,6,784,17,39556423,9560000,26536924,22579499,198427779940,47948201050,133282482240,113093498750,5016.322632,5022.529448,5015.502202,5008.680607,13019499.0
1,20220920,972,2,825,11,15743735,14105325,19488176,10360884,79997464760,71669944500,99008837700,52658571560,5081.225310,5080.456873,5081.055878,5082.440027,-3744441.0
2,20220919,1091,11,833,18,20431791,20790662,18169829,23052624,104187886880,105976653740,92602350880,117562189740,5099.302694,5096.489949,5097.319832,5099.731369,2261962.0
3,20220918,1345,7,965,10,18429944,16540907,21852464,13118387,94284926020,84494684770,111789046670,66990564120,5115.855264,5115.626625,5108.225611,5106.615937,-3422520.0
4,20220914,1987,8,983,9,35291055,9763396,35645122,9409329,182495693610,50332186110,183986143760,48841735960,5171.160046,5161.607913,5155.192528,5190.777787,-354067.0


In [30]:
foladClient = foladClient.iloc[::-1]
foladClient.reset_index(
    drop = True,
    inplace = True
)
foladClient['date'] = foladClient['date'].apply(
    lambda x: datetime.datetime.strptime(x, '%Y%m%d')
)
foladClient.head()

,date,individual_buy_count,corporate_buy_count,individual_sell_count,corporate_sell_count,individual_buy_vol,corporate_buy_vol,individual_sell_vol,corporate_sell_vol,individual_buy_value,corporate_buy_value,individual_sell_value,corporate_sell_value,individual_buy_mean_price,individual_sell_mean_price,corporate_buy_mean_price,corporate_sell_mean_price,individual_ownership_change
0,2008-11-26,14,2,4,1,106338,20010,126338,10,248937258,46844120,295757258,24120,2341.000000,2341.0,2341.035482,2412.000000,-20000.0
1,2008-11-29,116,175,3,2,4718258,33691903,121402,38288759,14980229312,107713006031,284080680,122409154663,3174.949168,2340.0,3196.999767,3196.999795,4596856.0
2,2008-11-30,10,2,1,1,87219,12791,100000,10,204005241,29918859,233900000,24100,2339.000000,2339.0,2339.055508,2410.000000,-12781.0
3,2008-12-01,8,1,5,1,128666,10,128666,10,300821108,24090,300821108,24090,2338.000000,2338.0,2409.000000,2409.000000,0.0
4,2008-12-02,7,1,1,0,18553,10000,28553,0,43358361,23370000,66728361,0,2337.000000,2337.0,2337.000000,NaN,-10000.0


In [31]:
marketIndex = download_financial_indexes(
    symbols = 'شاخص كل', 
    write_to_csv = True
)
marketIndex = marketIndex['شاخص كل']
marketIndex.rename(
    columns = {'value': 'TEDPIX'}, 
    inplace = True
)

print(f'Shape of TEDPIX: {marketIndex.shape}')
marketIndex.head()

Shape of TEDPIX: (3332, 2)


,TEDPIX,date
0,9248.9,2008-12-04
1,9248.9,2008-12-05
2,9178.3,2008-12-06
3,9130.5,2008-12-07
4,9089.2,2008-12-08


In [32]:
foladHist = pd.merge(
    foladHist, 
    marketIndex, 
    how = 'left', 
    on = 'date'
)
foladHist.head()

,date,open,high,low,adjClose,value,volume,count,yesterday,close,TEDPIX
0,2007-03-11,16.0,16.0,16.0,16.0,889437216900,468077431,7736,16.0,16.0,NaN
1,2007-03-12,16.0,16.0,16.0,16.0,193879458000,100041000,9214,16.0,16.0,NaN
2,2007-03-13,16.0,16.0,16.0,16.0,249241504527,126270939,5862,16.0,16.0,NaN
3,2007-03-14,16.0,16.0,16.0,16.0,51666379451,26705128,1901,16.0,16.0,NaN
4,2007-03-17,16.0,16.0,16.0,16.0,28239006789,14877283,1514,16.0,16.0,NaN


In [ ]:
# foladHist_new = pd.concat(
#     [
#         foladHist,
#         foladHist.shift(1).drop(columns = 'date').add_prefix('last_')
#     ], 
#     axis = 1
# )
# foladHist_new

In [ ]:
# foladHist['pct_change'] = foladHist['adjClose'].pct_change() * 100

In [ ]:
# foladHist.insert(1,'step', np.arange(1, len(foladHist) + 1))
# cols = ['date', 'jdate', 'step', 'TEDPIX',
#          'open', 'close', 'high', 'low', 'adjClose', 
#         'value', 'volume', 'count', 'yesterday']
# foladHist = foladHist[cols]

# foladHist.head()

# Indicators

In [6]:
emaCols = EMA(foladHist)
emaCols

,EMA_5d,last_EMA_5d,EMA_40d,last_EMA_40d,EMA_buy,EMA_position
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,16.000000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
3413,5055.984098,5068.976147,5254.501042,5266.013916,NaN,0.0
3414,5070.656065,5055.984098,5246.964406,5254.501042,NaN,0.0
3415,5107.104044,5070.656065,5243.697849,5246.964406,NaN,0.0
3416,5128.069362,5107.104044,5240.102832,5243.697849,NaN,0.0


In [ ]:
EMAzeroSig = emaCols['EMA_position'] == 0
EMAoneSig =  emaCols['EMA_position'] == 1

plt.figure(figsize = (20, 20))

plt.subplot(3, 1, 1)

plt.title('Plot based on EMA signals', fontsize = 15)
plt.plot(
    foladHist['date'], 
    np.log(foladHist['adjClose']),
    label = 'Price'
)
plt.scatter(
    foladHist['date'][EMAoneSig],
    np.log(foladHist['adjClose'][EMAoneSig]), 
    color = 'g',
    s = 50,
    label = 'Buy Position'
)
plt.scatter(
    foladHist['date'][EMAzeroSig],
    np.log(foladHist['adjClose'][EMAzeroSig]), 
    color = 'r',
    s = 50,
    label = 'Sell Position'
)
plt.legend(fontsize = 15)

plt.subplot(3, 1, 2)

plt.plot(
    foladHist['date'], 
    np.log(foladHist['adjClose']),
    label = 'Price'
)
plt.plot(
    foladHist['date'], 
    np.log(emaCols['EMA_40d']),
    label = '40-day EMA'
)
plt.plot(
    foladHist['date'], 
    np.log(emaCols['EMA_5d']),
    label = '5-day EMA'
)
plt.legend(fontsize = 15)

plt.subplot(3, 1, 3)

plt.plot(
    foladHist['date'], 
    foladHist['pct_change'].cumsum(),
    label = 'Passive investing (to percentage)'
)
plt.plot(
    foladHist['date'], 
    (foladHist['pct_change'] * emaCols['EMA_position']).cumsum(), 
    label = 'Active investing based on EMA signals (to percentage)'
)
plt.legend(fontsize = 15)

In [7]:
macdCols = MACD(foladHist)
macdCols

,MACD,MACD_diff,MACD_signal,MACD_buy,MACD_position
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
3413,-84.013710,-28.830609,-55.183100,NaN,0.0
3414,-80.193364,-20.008211,-60.185153,NaN,0.0
3415,-69.904564,-7.775529,-62.129035,NaN,0.0
3416,-61.844627,0.227527,-62.072154,1.0,0.0


In [ ]:
MACDzeroSig = macdCols['MACD_position'] == 0
MACDoneSig =  macdCols['MACD_position'] == 1

plt.figure(figsize = (20, 20))

plt.subplot(3, 1, 1)

plt.title('Plot based on MACD signals', fontsize = 15)
plt.plot(
    foladHist['date'], 
    np.log(foladHist['adjClose']),
    label = 'Price'
)
plt.scatter(
    foladHist['date'][MACDoneSig],
    np.log(foladHist['adjClose'][MACDoneSig]), 
    color = 'g',
    s = 50,
    label = 'Buy Position'
)
plt.scatter(
    foladHist['date'][MACDzeroSig],
    np.log(foladHist['adjClose'][MACDzeroSig]), 
    color = 'r',
    s = 50,
    label = 'Sell Position'
)
plt.legend(fontsize = 15)

plt.subplot(3, 1, 2)

plt.plot(
    foladHist['date'], 
    macdCols['MACD_signal'],
    label = 'MACD signal',
    color = 'red'
)

plt.bar(
    foladHist['date'], 
    macdCols['MACD'],
    label = '5-day EMA'
)
plt.legend(fontsize = 15)

plt.subplot(3, 1, 3)

plt.plot(
    foladHist['date'], 
    foladHist['pct_change'].cumsum(),
    label = 'Passive investing'
)
plt.plot(
    foladHist['date'], 
    (foladHist['pct_change'] * macdCols['MACD_position']).cumsum(), 
    label = 'Active investing based on MACD signals'
)
plt.legend(fontsize = 15)

In [8]:
stochasticCols = STOCHASTIC(foladHist)
stochasticCols

,STOCH_fast,STOCH_slow,last_STOCH_fast,last_STOCH_slow,STOCH_buy,STOCH_position
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
3413,9.302326,7.173640,10.869565,6.999922,NaN,0.0
3414,35.897436,13.257237,9.302326,7.173640,NaN,0.0
3415,58.974359,23.643658,35.897436,13.257237,NaN,0.0
3416,41.025641,31.213865,58.974359,23.643658,NaN,0.0


In [ ]:
STOCHzeroSig = stochasticCols['STOCH_position'] == 0
STOCHoneSig =  stochasticCols['STOCH_position'] == 1

plt.figure(figsize = (20, 20))

plt.subplot(3, 1, 1)

plt.title('Plot based on Stochastic signals', fontsize = 15)
plt.plot(
    foladHist['date'], 
    np.log(foladHist['adjClose']),
    label = 'Price'
)
plt.scatter(
    foladHist['date'][STOCHoneSig],
    np.log(foladHist['adjClose'][STOCHoneSig]), 
    color = 'g',
    s = 50,
    label = 'Buy Position'
)
plt.scatter(
    foladHist['date'][STOCHzeroSig],
    np.log(foladHist['adjClose'][STOCHzeroSig]), 
    color = 'r',
    s = 50,
    label = 'Sell Position'
)
plt.legend(fontsize = 15)

plt.subplot(3, 1, 2)

plt.plot(
    foladHist['date'], 
    stochasticCols['STOCH_fast'],
    label = 'Stochastic fast',
    color = 'blue'
)

plt.plot(
    foladHist['date'], 
    stochasticCols['STOCH_slow'],
    label = 'Stochastic slow',
    color = 'red'
)
plt.legend(fontsize = 15)

plt.subplot(3, 1, 3)

plt.plot(
    foladHist['date'], 
    foladHist['pct_change'].cumsum(),
    label = 'Passive investing'
)
plt.plot(
    foladHist['date'], 
    (foladHist['pct_change'] * macdCols['MACD_position']).cumsum(), 
    label = 'Active investing based on Stochastic signals'
)
plt.legend(fontsize = 15)

In [ ]:
bbCols = BB(foladHist_new)
bbCols

In [ ]:
BBzeroSig = bbCols['BB_position'] == 0
BBoneSig =  bbCols['BB_position'] == 1

plt.figure(figsize = (20, 20))

plt.subplot(3, 1, 1)

plt.title('Plot based on Bollinger Bands signals', fontsize = 15)
plt.plot(
    foladHist['date'], 
    np.log(foladHist['adjClose']),
    label = 'Price'
)
plt.scatter(
    foladHist['date'][BBoneSig],
    np.log(foladHist['adjClose'][BBoneSig]), 
    color = 'g',
    s = 10,
    label = 'Buy Position'
)
plt.scatter(
    foladHist['date'][BBzeroSig],
    np.log(foladHist['adjClose'][BBzeroSig]), 
    color = 'r',
    s = 10,
    label = 'Sell Position'
)
plt.legend(fontsize = 15)

plt.subplot(3, 1, 2)

plt.plot(
    foladHist['date'], 
    np.log(foladHist['adjClose']),
    label = 'Price'
)
plt.plot(
    foladHist['date'], 
    np.log(bbCols['BB_bbh']),
    label = 'Upper bands'
)
plt.plot(
    foladHist['date'], 
    np.log(bbCols['BB_bbl']),
    label = 'Lower bands'
)

plt.legend(fontsize = 15)

plt.subplot(3, 1, 3)

plt.plot(
    foladHist['date'], 
    foladHist['pct_change'].cumsum(),
    label = 'Passive investing'
)
plt.plot(
    foladHist['date'], 
    (foladHist['pct_change'] * bbCols['BB_position']).cumsum(), 
    label = 'Active investing based on Bollinger Bands signals'
)
plt.legend(fontsize = 15)

In [35]:
foladHist

,date,open,high,low,adjClose,value,volume,count,yesterday,close,...,SMA10d,SMA20d,last_SMA20d,SMA30d,SMA50d,last_SMA50d,SMA80d,RSI_buy,SMA_buy,SMA_position
0,2007-03-11,16.0,16.0,16.0,16.0,889437216900,468077431,7736,16.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-03-12,16.0,16.0,16.0,16.0,193879458000,100041000,9214,16.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-03-13,16.0,16.0,16.0,16.0,249241504527,126270939,5862,16.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-03-14,16.0,16.0,16.0,16.0,51666379451,26705128,1901,16.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-03-17,16.0,16.0,16.0,16.0,28239006789,14877283,1514,16.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3413,2022-09-11,5040.0,5060.0,5020.0,5030.0,189855163190,37750629,3062,5030.0,5030.0,...,5098.0,5274.0,5311.5,5269.966667,5257.42,5266.68,5387.8750,NaN,NaN,1.0
3414,2022-09-12,5100.0,5170.0,5050.0,5100.0,275376150730,53997996,5170,5030.0,5130.0,...,5085.0,5242.0,5274.0,5268.100000,5250.50,5257.42,5381.4875,NaN,0.0,1.0
3415,2022-09-13,5160.0,5230.0,5120.0,5180.0,400648290970,77278407,4958,5100.0,5220.0,...,5086.0,5217.5,5242.0,5269.866667,5245.18,5250.50,5376.5750,NaN,NaN,0.0
3416,2022-09-14,5250.0,5250.0,5120.0,5170.0,232827879720,45054451,3908,5180.0,5150.0,...,5089.0,5196.0,5217.5,5275.033333,5239.56,5245.18,5372.1875,NaN,NaN,0.0


In [34]:
smaCols = SMA(foladHist)
smaCols

,SMA10d,SMA20d,last_SMA20d,SMA30d,SMA50d,last_SMA50d,SMA80d,SMA_buy,SMA_position
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3413,5098.0,5274.0,5311.5,5269.966667,5257.42,5266.68,5387.8750,NaN,1.0
3414,5085.0,5242.0,5274.0,5268.100000,5250.50,5257.42,5381.4875,0.0,1.0
3415,5086.0,5217.5,5242.0,5269.866667,5245.18,5250.50,5376.5750,NaN,0.0
3416,5089.0,5196.0,5217.5,5275.033333,5239.56,5245.18,5372.1875,NaN,0.0


In [ ]:
SMAzeroSig = smaCols['SMA_position'] == 0
SMAoneSig =  smaCols['SMA_position'] == 1

plt.figure(figsize = (20, 20))

plt.subplot(3, 1, 1)

plt.title('Plot based on Simple Moving Average signals', fontsize = 15)
plt.plot(
    foladHist['date'], 
    np.log(foladHist['adjClose']),
    label = 'Price'
)
plt.scatter(
    foladHist['date'][SMAoneSig],
    np.log(foladHist['adjClose'][SMAoneSig]), 
    color = 'g',
    s = 10,
    label = 'Buy Position'
)
plt.scatter(
    foladHist['date'][SMAzeroSig],
    np.log(foladHist['adjClose'][SMAzeroSig]), 
    color = 'r',
    s = 10,
    label = 'Sell Position'
)
plt.legend(fontsize = 15)

plt.subplot(3, 1, 2)

plt.plot(
    foladHist['date'], 
    np.log(foladHist['adjClose']),
    label = 'Price'
)
plt.plot(
    foladHist['date'], 
    np.log(smaCols['SMA20d']),
    label = 'SMA20d'
)
plt.plot(
    foladHist['date'], 
    np.log(smaCols['SMA80d']),
    label = 'SMA80d'
)

plt.legend(fontsize = 15)

plt.subplot(3, 1, 3)

plt.plot(
    foladHist['date'], 
    foladHist['pct_change'].cumsum(),
    label = 'Passive investing'
)
plt.plot(
    foladHist['date'], 
    (foladHist['pct_change'] * smaCols['SMA_position']).cumsum(), 
    label = 'Active investing based on Simple Moving Average signals'
)
plt.legend(fontsize = 15)

In [10]:
rsiCols = RSI(foladHist)
rsiCols

,RSI,RSI_buy,RSI_position
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
3413,34.793150,NaN,0.0
3414,41.220239,NaN,0.0
3415,47.550449,NaN,0.0
3416,46.874367,NaN,0.0


In [ ]:
RSIzeroSig = rsiCols['RSI_position'] == 0
RSIoneSig =  rsiCols['RSI_position'] == 1

plt.figure(figsize = (20, 20))

plt.subplot(3, 1, 1)

plt.title('Plot based on Relative Strength Index signals', fontsize = 15)
plt.plot(
    foladHist['date'], 
    np.log(foladHist['adjClose']),
    label = 'Price'
)
plt.scatter(
    foladHist['date'][RSIoneSig],
    np.log(foladHist['adjClose'][RSIoneSig]), 
    color = 'g',
    s = 50,
    label = 'Buy Position'
)
plt.scatter(
    foladHist['date'][RSIzeroSig],
    np.log(foladHist['adjClose'][RSIzeroSig]), 
    color = 'r',
    s = 50,
    label = 'Sell Position'
)
plt.legend(fontsize = 15)

plt.subplot(3, 1, 2)

plt.plot(
    foladHist['date'], 
    rsiCols['RSI'],
    label = 'RSI',
    color = 'blue'
)

plt.legend(fontsize = 15)

plt.subplot(3, 1, 3)

plt.plot(
    foladHist['date'], 
    foladHist['pct_change'].cumsum(),
    label = 'Passive investing'
)
plt.plot(
    foladHist['date'], 
    (foladHist['pct_change'] * rsiCols['RSI_position']).cumsum(), 
    label = 'Active investing based on Relative Strength Index signals'
)
plt.legend(fontsize = 15)

# Shabnam and Mohammad' work!

In [11]:
candle_based(foladHist)
candle_based

KeyError: 0

In [ ]:
retCols = ret_based_func(foladHist)
retCols

In [ ]:
prpCols = prp_based_func(foladHist)
prpCols

In [ ]:
wghtCols = wght_based_feature(foladHist)
wghtCols

In [ ]:
calCols = calender_features(foladHist)
calCols

# Concat

In [ ]:
foladDf = pd.concat(
    [
        foladHist,
        emaCols,
        macdCols,
        bbCols,
        smaCols,
        rsiCols,
        retCols, # shabnam
        prpCols,
        wghtCols       
    ],
    axis = 1
) # 3412 rows × 53 columns

# foladDf.dropna(
#     axis = 'columns', 
#     thresh = len(foladDf) - 81,
#     inplace = True
# ) # 3412 rows × 43 columns

foladDf.dropna(
    axis = 'index', 
    how = 'any',
    inplace = True
) # 3333 rows × 40 columns

foladDf.head()

In [ ]:
pivots = pd.DataFrame(
    peak_valley_pivots(

        foladHist['adjClose'], 
        0.08, 
        -0.08
    ) * -1,
    columns = ['label']
)

signals = pivots.replace(to_replace = 0, value = np.nan)
signals.fillna(method = 'ffill', inplace = True)
signals.replace(to_replace = -1, value = 0,  inplace = True)

np.unique(signals, return_counts = True)

In [ ]:
zeroSig = signals[-len(foladDf):] == 0
oneSig = signals[-len(foladDf):] == 1

plt.figure(figsize = (25, 10))
plt.title('Plot based on ZigZag', fontsize = 25)

plt.plot(
    foladDf['date'], 
    np.log(foladDf['adjClose'])
)

plt.scatter(
    foladDf['date'][np.squeeze(oneSig.values)],
    np.log(foladDf['adjClose'][np.squeeze(oneSig.values)]), 
    color = 'g',
    s = 50,
    label = 'Buy Position'
)

plt.scatter(
    foladDf['date'][np.squeeze(zeroSig.values)],
    np.log(foladDf['adjClose'][np.squeeze(zeroSig.values)]), 
    color = 'r',
    s = 50,
    label = 'Sell Position'
)
plt.legend(fontsize = 25)

# model

In [ ]:
y = signals[-len(foladDf):]

X_train, X_test, y_train, y_test = train_test_split(
    foladDf.drop(columns = ['date']).values, 
    y, 
    shuffle = False, 
    random_state = 0)

In [ ]:
from model import RFClf
trainPred, testPred, trainProb, testProb, featureImport, trainClassReport, testClassReport \
    = RFClf(X_train, y_train, X_test, y_test)

In [ ]:
print(f'Training classification report:\n{trainClassReport}')
print(f'Testing classification report:\n{testClassReport}')

In [ ]:
Xdf_train = foladDf[:len(X_train)]
Xdf_test = foladDf[len(X_train):]
ydf_train = y[:len(X_train)]
ydf_test = y[len(X_train):]

zeroSig_test = ydf_test == 0
oneSig_test = ydf_test == 1

zeroSig_testPred = testPred == 0
oneSig_testPred = testPred == 1

In [ ]:
plt.figure(figsize = (25, 25))

plt.subplot(3, 1, 1)
plt.title('Plot based on labels on truth data', fontsize = 15)
plt.plot(
    Xdf_test['date'], 
    np.log(Xdf_test['adjClose'])
)
plt.scatter(
    Xdf_test['date'][np.squeeze(oneSig_test.values)],
    np.log(Xdf_test['adjClose'][np.squeeze(oneSig_test.values)]), 
    color = 'g',
    s = 50, 
    label = 'Buy Position'
)
plt.scatter(
    Xdf_test['date'][np.squeeze(zeroSig_test.values)],
    np.log(Xdf_test['adjClose'][np.squeeze(zeroSig_test.values)]), 
    color = 'r',
    s = 50,
    label = 'Sell Position'
)
plt.legend(fontsize = 15)

plt.subplot(3, 1, 2)
plt.title('Plot based on predictions on test data', fontsize = 15)

plt.plot(
    Xdf_test['date'], 
    np.log(Xdf_test['adjClose'])
)
plt.scatter(
    Xdf_test['date'][np.squeeze(oneSig_testPred)],
    np.log(Xdf_test['adjClose'][np.squeeze(oneSig_testPred)]), 
    color = 'g',
    s = 50, 
    label = 'Buy Position'
)
plt.scatter(
    Xdf_test['date'][np.squeeze(zeroSig_testPred)],
    np.log(Xdf_test['adjClose'][np.squeeze(zeroSig_testPred)]), 
    color = 'r',
    s = 50,
    label = 'Sell Position'
)
plt.legend(fontsize = 15)

# plt.subplot(3, 1, 3)
# plt.plot(
#     foladHist['date'], 
#     foladHist['pct_change'].cumsum(),
#     label = 'Ideal'
# )
# plt.plot(
#     foladHist['date'], 
#     (foladHist['pct_change'] * rsiCols['RSI_position']).cumsum(), 
#     label = 'Active investing based on model signals'
# )

# plt.legend(fontsize = 15)

In [ ]:
plt.figure(figsize = (10, 20))
plt.barh(foladDf.drop(columns = ['date']).columns, featureImport)

# Data pre-processing

In [ ]:
# foladHist.insert(1,'step', np.arange(1, len(foladHist) + 1))
# foladHist = pd.merge(foladHist, marketIndex, how = 'left', on = 'date')

# cols = ['date', 'jdate', 'step', 'TEDPIX',
#         'open', 'close', 'high', 'low', 'adjClose', 
#         'value', 'volume', 'count', 'yesterday']
# foladHist = foladHist[cols]

# foladHist.head()

In [ ]:
# foladClient['date'] = pd.to_datetime(foladClient['date'])
# foladClient.iloc[:, 1: 13] = foladClient.iloc[:, 1: 13].astype('float')

# foladClient['Enter_individual_money'] = foladClient['individual_buy_value'] - foladClient['individual_sell_value']
# foladClient['Individual_buy_per_capita'] = foladClient['individual_buy_value'] / foladClient['individual_buy_count']
# foladClient['Individual_sell_per_capita'] = foladClient['individual_sell_value'] / foladClient['individual_sell_count']
# foladClient['Individual_buy_power_to_Individual_sell'] = foladClient['Individual_buy_per_capita'] / foladClient['Individual_sell_per_capita']

# foladClient.head()

In [ ]:
# foladHist = pd.merge(foladHist, foladClient.iloc[:, [0, -4, -3, -2, -1]], how = 'left', on = 'date')
# foladHist.head()

In [ ]:
# foladHist['jdate'] = foladHist['date'].jalali.to_jalali()

# foladHist['year'] = foladHist['jdate'].jalali.year
# foladHist['month'] = foladHist['jdate'].jalali.month
# foladHist['quarter'] = foladHist['jdate'].jalali.quarter
# foladHist['day'] = foladHist['jdate'].jalali.day
# foladHist['weekday'] = foladHist['jdate'].jalali.weekday

# foladHist

# Candel Plot

In [ ]:
# fig = make_subplots(
#     rows = 2, 
#     cols = 1, 
#     shared_xaxes = True, 
#     vertical_spacing = 0.03, 
#     subplot_titles = ('OHLC', 'Volume'), 
#     row_width = [0.2, 0.7]
# )

# fig.add_trace(
#     go.Candlestick(
#         x = foladHist['date'],
#         open = foladHist['open'],
#         high = foladHist['high'],
#         low = foladHist['low'],
#         close = foladHist['adjClose']
#     ),
#     row = 1, col = 1
# )

# fig.add_trace(
#     go.Bar(
#         x = foladHist['date'],
#         y = foladHist['volume'], 
#         showlegend = False
#         ),
#         row = 2, col = 1
# )
# fig.update(layout_xaxis_rangeslider_visible = False)
# fig.show()